In [95]:
import time
from datetime import datetime
import pytz
import os
import sys
import numpy as np
import matplotlib
%matplotlib ipympl
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style

style.use('ggplot')

import multiprocessing
from multiprocessing import Pool

from pathlib import Path

import photutils
import astropy.units as u
from astropy import stats
from astropy.io import fits
from astropy.table import Table, hstack, vstack

from astropy.modeling.models import Gaussian2D, Polynomial2D, Moffat2D
from astropy.modeling.fitting import LevMarLSQFitter

from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT
from mmtwfs.custom_exceptions import WFSAnalysisFailed

tz = pytz.timezone("America/Phoenix")

In [2]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [3]:
hdr_keys = ['DATE-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'TEMP', 'FOCUS', 'CHAM_DPT', 'CHAM_T', 'OUT_T', 'WIND_W', 'WDIR_W', 'PRESSURE',
            'EXPTIME', 'AIRMASS', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']
f5_hdr_keys = ['DATEOBS', 'UT', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'FOCUS', 'T_OUT', 'T_CHAM', 'RH_OUT', 'RH_CHAM', 'P_BARO', 'EXPTIME', 'AIRMASS',
               'WIND', 'WINDDIR', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']
old_f9_keys = ['DATE-OBS', 'TIME-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'FOCUS', 'T_OUT', 'T_CHAM', 'RH_OUT', 'RH_CHAM', 'P_BARO', 'EXPTIME', 'AIRMASS',
               'WIND', 'WINDDIR', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY']

In [106]:
def check_image(f, wfskey=None):
    hdr = {}
    with fits.open(f) as hdulist:
        for h in hdulist:
            hdr.update(h.header)
        data = hdulist[-1].data
    
    # if wfskey is None, figure out which WFS from the header info...
    if wfskey is None:
        # check for MMIRS
        if 'WFSNAME' in hdr:
            if 'mmirs' in hdr['WFSNAME']:
                wfskey = 'mmirs'
        if 'mmirs' in f.name:
            wfskey = 'mmirs'

        # check for binospec
        if 'bino' in f.name:
            wfskey = 'binospec'
        if 'ORIGIN' in hdr:
            if 'Binospec' in hdr['ORIGIN']:
                wfskey = 'binospec'

        # check for new F/9
        if 'f9wfs' in f.name:
            wfskey = 'newf9'
        if 'OBSERVER' in hdr:
            if 'F/9 WFS' in hdr['OBSERVER']:
                wfskey = 'newf9'
        if wfskey is None and 'CAMERA' in hdr:
            if 'F/9 WFS' in hdr['CAMERA']:
                wfskey = 'newf9'

        # check for old F/9
        if 'INSTRUME' in hdr:
            if 'Apogee' in hdr['INSTRUME']:
                wfskey = 'oldf9'
        if 'DETECTOR' in hdr:
            if 'Apogee' in hdr['DETECTOR']:
                wfskey = 'oldf9'

        # check for F/5 (hecto)
        if wfskey is None and 'SEC' in hdr:  # mmirs has SEC in header as well and is caught above
            if 'F5' in hdr['SEC']:
                wfskey = 'f5'
        if Path(f.parent / "F5").exists():
            wfskey = 'f5'
            
    if wfskey is None:
        # if wfskey is still None at this point, whinge.
        print(f"Can't determine WFS for {f.name}...")

    if 'AIRMASS' not in hdr:
        if 'SECZ' in hdr:
            hdr['AIRMASS'] = hdr['SECZ']
        else:
            hdr['AIRMASS'] = np.nan

    # we need to fix the headers in all cases to have a proper DATE-OBS entry with
    # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
    # for analysis in pandas.
    dtime = None
    if 'DATEOBS' in hdr:
        dateobs = hdr['DATEOBS']
        if 'UT' in hdr:
            ut = hdr['UT']
        elif 'TIME-OBS' in hdr:
            ut = hdr['TIME-OBS']
        else:
            ut = "07:00:00"  # midnight
        timestring = dateobs + " " + ut + " UTC"
        if wfskey in ('newf9', 'f5'):
            if '-' in timestring:
                dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
            else:
                dtime = datetime.strptime(timestring, "%a %b %d %Y %H:%M:%S %Z")

        else:
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
    else:
        if wfskey == "oldf9":
            d = hdr['DATE-OBS']
            if '/' in d:
                day, month, year = d.split('/')
                year = str(int(year) + 1900)
                timestring = year + "-" + month + "-" + day + " " + hdr['TIME-OBS'] + " UTC"
            else:
                timestring = d + " " + hdr['TIME-OBS'] + " UTC"
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            if 'DATE-OBS' in hdr:
                timestring = hdr['DATE-OBS'] + " UTC"
                try:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S.%f %Z")
                except:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S %Z")
            else:
                dt = datetime.fromtimestamp(f.stat().st_ctime)
                local_dt = tz.localize(dt)
                dtime = local_dt.astimezone(pytz.utc)

    if dtime is None:
        print(f"No valid timestamp in header for {f.name}...")
        obstime = None
    else:
        obstime = dtime.isoformat().replace('+00:00', '')
        
    hdr['WFSKEY'] = wfskey
    hdr['OBS-TIME'] = obstime
    return data, hdr

In [62]:
findpars = {
    "oldf9": {"fwhm": 8.0, "thresh": 5.0},
    "newf9": {"fwhm": 12.0, "thresh": 7.0},
    "f5": {"fwhm": 9.0, "thresh": 5.0},
    "mmirs": {"fwhm": 7.0, "thresh": 5.0},
    "binospec": {"fwhm": 7.0, "thresh": 5.0}
}

In [6]:
#%cd /Users/tim/MMT/wfsdat/20180209
%cd /Volumes/LACIE\ SHARE/wfsdat

/Volumes/LACIE SHARE/wfsdat


In [6]:
file = Path("wfs_start_0188.fits")
data, hdr = check_image(file)
data.shape, hdr

((512, 512),
 {'SIMPLE': True,
  'BITPIX': 16,
  'NAXIS': 2,
  'NAXIS1': 512,
  'NAXIS2': 512,
  'BZERO': 32768,
  'BSCALE': 1,
  'OFFSET1': 0,
  'OFFSET2': 0,
  'XFACTOR': 1,
  'YFACTOR': 1,
  'EXPTIME': 20,
  'ORIGIN': 'Clear Sky Institute',
  'TELESCOP': 'Engineering model',
  'CDELT1': -0.0003444444444,
  'CDELT2': -0.0003444444444,
  'INSTRUME': 'Apogee CCD Camera',
  'JD': 2452720.70917,
  'DATE-OBS': '22/03/103',
  'TIME-OBS': '05:01:12',
  'FILTER': 'C',
  'CAMTEMP': -19,
  'AIRMASS': nan,
  'WFSKEY': 'oldf9',
  'OBS-TIME': '2003-03-22T05:01:12'})

In [125]:
def process_image(f, clobber=False):
    tablefile = Path(str(f.parent / f.stem) + ".csv")
    if tablefile.exists() and not clobber:
        print(f"{f.name} already processed...")
        return
    else:
        print(f"Processing {f.name}...")

    try:
        data, hdr = check_image(f)
        w = hdr['WFSKEY']
        mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, iters=None)
        data = data - median
        spots, fig = wfsfind(data, fwhm=findpars[w]['fwhm'], threshold=findpars[w]['thresh'], plot=False)
    except WFSAnalysisFailed as e:
        try:
            spots, fig = wfsfind(data, fwhm=2.*findpars[w]['fwhm'], threshold=findpars[w]['thresh'], plot=False)
        except Exception as e:
            print(f"Failed to find spots for {str(Path(str(f.parent / f.stem)))}: {e}")
            return
    except Exception as ee:
        print(f"Failure in data loading or spot finding in {str(Path(str(f.parent / f.stem)))}: {ee}")
        return      

    apsize = 15.
    apers = photutils.CircularAperture(
            (spots['xcentroid'], spots['ycentroid']),
            r=apsize
    )
    masks = apers.to_mask(method="subpixel")
    props = []
    spot_lines = []
    fit_lines = []
    for m, s in zip(masks, spots):
        tline = {}
        subim = m.cutout(data)
        try:
            props_table = photutils.data_properties(subim).to_table()
        except Exception as e:
            print(f"Can't measure source properties for {str(Path(str(f.parent / f.stem)))}: {e}")
            continue
        moment_fwhm = 0.5 * (props_table['semimajor_axis_sigma'][0].value + props_table['semiminor_axis_sigma'][0].value) * stats.gaussian_sigma_to_fwhm
        props.append(props_table)
        spot_lines.append(s)
        tline['filename'] = f.name
        for k in hdr:
            if 'COMMENT' not in k and k != '':
                tline[k] = hdr[k]

        y, x = np.mgrid[:subim.shape[0], :subim.shape[1]]
        sigma = (props_table['semimajor_axis_sigma'][0].value + props_table['semiminor_axis_sigma'][0].value) / 2.

        fitter = LevMarLSQFitter()
        gauss_model = Gaussian2D(
            amplitude=subim.max(),
            x_mean=subim.shape[1]/2.,
            y_mean=subim.shape[0]/2.,
            x_stddev = sigma,
            y_stddev = sigma
        ) + Polynomial2D(degree=0)
        moffat_model = Moffat2D(
            amplitude=subim.max(),
            x_0=subim.shape[1]/2.,
            y_0=subim.shape[0]/2.,
            gamma=sigma
        ) + Polynomial2D(degree=0)
        
        try:
            gauss_fit = fitter(gauss_model, x, y, subim)
            gauss_resid = subim - gauss_fit(x, y)
            gauss_fwhm = 0.5 * (gauss_fit.x_stddev_0.value + gauss_fit.y_stddev_0.value) * stats.gaussian_sigma_to_fwhm
            tline['gauss_x'] = gauss_fit.x_mean_0.value
            tline['gauss_y'] = gauss_fit.y_mean_0.value
            tline['gauss_sigx'] = gauss_fit.x_stddev_0.value
            tline['gauss_sigy'] = gauss_fit.y_stddev_0.value
            tline['gauss_amplitude'] = gauss_fit.amplitude_0.value
            tline['gauss_theta'] = gauss_fit.theta_0.value
            tline['gauss_background'] = gauss_fit.c0_0_1.value
            tline['gauss_rms'] = np.nanstd(gauss_resid)
            tline['gauss_fwhm'] = gauss_fwhm
        except Exception as e:
            print(f"Gaussian fit failed in {str(Path(str(f.parent / f.stem)))}: {e}")
            tline['gauss_x'] = np.nan
            tline['gauss_y'] = np.nan
            tline['gauss_sigx'] = np.nan
            tline['gauss_sigy'] = np.nan
            tline['gauss_amplitude'] = np.nan
            tline['gauss_theta'] = np.nan
            tline['gauss_background'] = np.nan
            tline['gauss_rms'] = np.nan
            tline['gauss_fwhm'] = np.nan
            
        try:
            moffat_fit = fitter(moffat_model, x, y, subim)
            moffat_resid = subim - moffat_fit(x, y)
            gamma = moffat_fit.gamma_0.value
            alpha = moffat_fit.alpha_0.value
            moffat_fwhm = np.abs(2. * gamma * np.sqrt(2.**(1./alpha) - 1.))
            tline['moffat_amplitude'] = moffat_fit.amplitude_0.value
            tline['moffat_gamma'] = gamma
            tline['moffat_alpha'] = alpha
            tline['moffat_x'] = moffat_fit.x_0_0.value
            tline['moffat_y'] = moffat_fit.y_0_0.value
            tline['moffat_background'] = moffat_fit.c0_0_1.value
            tline['moffat_rms'] = np.nanstd(moffat_resid)
            tline['moment_fwhm'] = moment_fwhm
            tline['moffat_fwhm'] = moffat_fwhm
        except Exception as e:
            print(f"Moffat fit failed in {str(Path(str(f.parent / f.stem)))}: {e}")
            tline['moffat_amplitude'] = np.nan
            tline['moffat_gamma'] = np.nan
            tline['moffat_alpha'] = np.nan
            tline['moffat_x'] = np.nan
            tline['moffat_y'] = np.nan
            tline['moffat_background'] = np.nan
            tline['moffat_rms'] = np.nan
            tline['moment_fwhm'] = np.nan
            tline['moffat_fwhm'] = np.nan
        fit_lines.append(tline)
        
    fit_table = Table(fit_lines)
    spot_table = Table(vstack(spot_lines))
    prop_table = Table(vstack(props))
    t = hstack([spot_table, prop_table, fit_table])
    try:
        if tablefile.exists():
            if clobber:
                t.write(tablefile, format="csv")
        else:
            t.write(tablefile, format="csv")
    except Exception as e:
        print(f"Failed to write {str(tablefile)}: {e}")
    return t

In [37]:
process_image(file, clobber=True)

Processing wfs_start_0188.fits...


In [93]:
t.hist(column='gauss_fwhm', bins=50, range=(0,10), alpha=0.6)
t.hist(column='moffat_fwhm', bins=50, range=(0,10), alpha=0.6)
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [86]:
t['gauss_background'].mean(), t['moffat_background'].mean()

(495.81356566699588, 398.06243108166296)

In [87]:
t.hist(column='ellipticity', bins=50, range=(0,1.0), alpha=0.6)

FigureCanvasNbAgg()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1c217115f8>]], dtype=object)

In [97]:
(t['moffat_rms']/t['moffat_amplitude']).mean()

0.015914200328812602

In [91]:
%matplotlib ipympl
plt.imshow(gauss_resids[100])
plt.show()

FigureCanvasNbAgg()

In [22]:
#%matplotlib ipympl
plt.imshow(moffat_resids[10])
plt.show()

In [35]:
%lprun -f process_image process_image(file)

Timer unit: 1e-06 s

Total time: 29.8775 s
File: <ipython-input-33-bb5c1b33ae24>
Function: process_image at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def process_image(f, clobber=False):
     2         1         91.0     91.0      0.0      tablefile = Path(f.stem + ".csv")
     3         1         90.0     90.0      0.0      if tablefile.exists() and not clobber:
     4                                                   print(f"{file.name} already processed...")
     5                                                   return
     6                                           
     7         1     150092.0 150092.0      0.5      data, hdr = check_image(file)
     8         1     510791.0 510791.0      1.7      mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, iters=None)
     9         1        272.0    272.0      0.0      data = data - median
    10         1     322845.0 322845.0      1.1      s

In [129]:
rootdir = Path("/Volumes/LACIE SHARE/wfsdat")
dirs = sorted(list(rootdir.glob("200312*")))

In [130]:
nproc = 8
for d in dirs:
    if d.is_dir():
        print(f"Working in {d.name}...")
        fitsfiles = sorted(list(d.glob("*.fits")))
        with Pool(processes=nproc) as pool:  # my mac's i7 has 4 cores + hyperthreading so 8 virtual cores. 
            pool.map(process_image, fitsfiles)  # plines comes out in same order as fitslines!

Working in 20031205...
Processing test.fits...
Can't determine WFS for test.fits...
Failure in data loading or spot finding in /Volumes/LACIE SHARE/wfsdat/20031205/test: None
Working in 20031206...
Processing elcoll_el30_0004.fits...
Processing elcoll_el20_coll2_0000.fits...
Processing elcoll_el35_0004.fits...
Processing back.fits...
Processing elcoll_el25_0002.fits...
Processing elcoll_el40_0002.fits...
Processing elcoll_el45_0002.fits...
Processing elcoll_el50_0002.fits...


Processing center_0000.fits...


Processing elcoll_el35_0005.fits...
Processing elcoll_el25_coll_0000.fits...
Processing elcoll_el45_0003.fits...
Processing elcoll_el20_coll2_0001.fits...
Processing elcoll_el50_0003.fits...
Processing elcoll_el40_0003.fits...
Processing elcoll_el30_coll_0000.fits...
Processing elcoll_el20_0000.fits...
Processing elcoll_el35_coll_0000.fits...
Processing elcoll_el30_0000.fits...
Processing elcoll_el45_0004.fits...
Processing elcoll_el20_coll2_0002.fits...
Processing elcoll_el50_0004.fits...
Processing elcoll_el40_0004.fits...
Processing elcoll_el20_0001.fits...
Processing elcoll_el35_0000.fits...
Processing elcoll_el35_0001.fits...
Processing elcoll_el35_0002.fits...
Processing elcoll_el35_coll_0001.fits...
Processing elcoll_el45_coll_0000.fits...
Processing elcoll_el20_coll_0000.fits...
Processing elcoll_el20_0002.fits...
Processing elcoll_el30_0001.fits...
Processing elcoll_el50_coll_0000.fits...
Processing elcoll_el40_coll_0000.fits...
Processing elcoll_el35_0003.fits...
Processing e

Processing mid_0008.fits...
Processing mid_0009.fits...
Processing mid_0010.fits...
Processing start_0000.fits...
Processing start_0001.fits...
Processing start_0002.fits...
Processing start_0003.fits...
Processing start_0004.fits...
Processing test.fits...
Working in 20031210...
Processing back.fits...
Processing start_0014.fits...
Processing start_0002.fits...
Processing start_0008.fits...
Processing start_0011.fits...
Processing start_0005.fits...
Processing start_0017.fits...
Processing start_0020.fits...


Processing start_0000.fits...


...

Processing focus1_corr1_0000.fits...


Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0003: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0006.fits...
Processing focus1_corr1_0004.fits...
Processing focus1_start_0000.fits...
Processing focus1_start_0002.fits...
Processing focus2_start_0001.fits...
Processing focus1_corr1_0002.fits...
Processing focus2_start_0005.fits...


Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0001: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0008.fits...
Processing focus2_start_0010.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0008: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0009.fits...


Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0009: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0012.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0010: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0011.fits...
Processing focus2_start_0007.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0012: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0013.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0011: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing focus2_start_0014.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0013: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing start_0001.fits...
Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/focus2_start_0014: (WFSAnalysisFailed(...), 'No WFS spots detec

Failed to find spots for /Volumes/LACIE SHARE/wfsdat/20031223/test: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Processing start_corr5_0000.fits...
Processing start_corr4_0001.fits...
Processing start_corr5_0002.fits...


In [47]:
test = rootdir / "20030321" / "end_0000.fits"

In [54]:
str(test.parent / test.stem) + ".csv"

'/Volumes/LACIE SHARE/wfsdat/20030321/end_0000.csv'

In [116]:
test, fig = wfsfind("20031019/start_0001.fits", fwhm=8.)

FigureCanvasNbAgg()

In [120]:
test, hdr = check_image(Path("20031019/start_0001.fits"))
mean, median, stddev = stats.sigma_clipped_stats(test, sigma=3.0, iters=None)

In [117]:
test_table = process_image(Path("20031019/start_0001.fits"))

Processing start_0001.fits...
Failed to write 20031019/start_0001.csv: unhashable type: 'list'


In [87]:
tstfile = Path("20030517/wfs0029.fits")

In [99]:
dt = datetime.fromtimestamp(tstfile.stat().st_ctime)
local_dt = tz.localize(dt)
utc_dt = local_dt.astimezone(pytz.utc).isoformat().replace('+00:00', '')
utc_dt

'2003-05-18T06:49:02'

In [118]:
test_table

id_1,xcentroid_1,ycentroid_1,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag,id_2,xcentroid_2,ycentroid_2,sky_centroid,sky_centroid_icrs,source_sum,source_sum_err,background_sum,background_mean,background_at_centroid,xmin,xmax,ymin,ymax,min_value,max_value,minval_xpos,minval_ypos,maxval_xpos,maxval_ypos,area,equivalent_radius,perimeter,semimajor_axis_sigma,semiminor_axis_sigma,eccentricity,orientation,ellipticity,elongation,covar_sigx2,covar_sigxy,covar_sigy2,cxx,cxy,cyy,col0 [22],AIRMASS,BITPIX,DATE,EXTEND,IRAF-TLM,LTM1_1,LTM2_2,LTV1,LTV2,NAXIS,NAXIS1,NAXIS2,OBS-TIME,ORIGIN,SIMPLE,WFSKEY,filename,gauss_amplitude,gauss_background,gauss_fwhm,gauss_rms,gauss_sigx,gauss_sigy,gauss_theta,gauss_x,gauss_y,moffat_alpha,moffat_amplitude,moffat_background,moffat_fwhm,moffat_gamma,moffat_rms,moffat_x,moffat_y,moment_fwhm
,,,,,,,,,,,,pix,pix,,,,,,,,pix,pix,pix,pix,,,pix,pix,pix,pix,pix2,pix,pix,pix,pix,,rad,,,pix2,pix2,pix2,1 / pix2,1 / pix2,1 / pix2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,object,object,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,int64,str19,bool,str19,float64,float64,float64,float64,int64,int64,int64,str19,str37,bool,str2,str15,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,3.10484092468,251.015965297,0.304448351128,0.264250925016,0.4908556983,121.0,0.0,1894.0,1.96380127688,-0.732743845184,1,19.3889715998,9.78050869542,None,None,495031.0,None,None,None,None,0.0,30.0,0.0,30.0,0.0,2093.0,0.0,0.0,12.0,18.0,961.0,17.48987709,120.0,5.75752114939,5.24886484916,0.410959516616,1.53323081407,0.0883464058632,1.09690786767,27.5584788661,0.210111506495,33.1411531244,0.036288227078,-0.000460127264177,0.0301754237477,..,nan,32,2010-06-19T18:33:56,False,2010-06-19T18:33:55,0.25,0.25,0.375,0.375,2,258,256,2010-06-19T21:33:56,NOAO-IRAF FITS Image Kernel July 2003,True,f5,start_0001.fits,2010.68672117,-140.506549996,18.8425111169,370.386620267,6.90556396933,9.09779149446,-0.0017121280301,18.82271513,8.6080931914,14.4453862422,2014.93710518,-172.730973883,18.9481469917,42.7324675539,384.204388277,19.1968963198,9.29030285389,12.9590291864


In [121]:
hdr

{'SIMPLE': True,
 'BITPIX': 32,
 'NAXIS': 2,
 'NAXIS1': 258,
 'NAXIS2': 256,
 'EXTEND': False,
 'ORIGIN': 'NOAO-IRAF FITS Image Kernel July 2003',
 'DATE': '2010-06-19T18:33:56',
 'IRAF-TLM': '2010-06-19T18:33:55',
 'LTM1_1': 0.25,
 'LTM2_2': 0.25,
 'LTV1': 0.375,
 'LTV2': 0.375,
 '': 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ,
 'AIRMASS': nan,
 'WFSKEY': 'f5',
 'OBS-TIME': '2010-06-19T21:33:56'}